In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from tqdm.auto import tqdm

from numpy.random import seed
import numpy.matlib
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
from keras import backend as K

from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder, StandardScaler
from sklearn.preprocessing import QuantileTransformer

def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true)))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
train = pd.read_pickle('./input/train_248.pkl')
test = pd.read_pickle('./input/test_247.pkl')

In [4]:
colNames = [col for col in list(train.columns)
            if col not in {"stock_id", "time_id", "target", "row_id"}]
len(colNames)

244

In [5]:
# kfold based on the knn++ algorithm
out_train = pd.read_csv('./input/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')
#out_train[out_train.isna().any(axis=1)]
out_train = out_train.fillna(out_train.mean())
out_train.head()

# code to add the just the read data after first execution

# data separation based on knn ++
nfolds = 5 # number of folds
index = []
totDist = []
values = []
# generates a matriz with the values of 
mat = out_train.values

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds) # number of individuals

# adds index in the last column
mat = np.c_[mat,np.arange(mat.shape[0])]


lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)

lineNumber = np.sort(lineNumber)[::-1]

for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

# saves index
for n in range(nfolds):
    values.append([lineNumber[n]])    

s=[]
for n in range(nfolds):
    s.append(mat[lineNumber[n],:])
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in range(nind-1):    

    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
         # saves the values of index           

        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(nfolds):
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)

for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

In [6]:
out_train

stock_id,0,1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,46,47,48,50,51,52,53,55,56,58,59,60,61,62,63,64,66,67,68,69,70,72,73,74,75,76,77,78,80,81,82,83,84,85,86,87,88,89,90,93,94,95,96,97,98,99,100,101,102,103,104,105,107,108,109,110,111,112,113,114,115,116,118,119,120,122,123,124,125,126
time_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,0.004136,0.006340,0.001848,0.005300,0.004468,0.006234,0.007651,0.003624,0.010036,0.007291,0.005707,0.005918,0.002148,0.002627,0.005091,0.005485,0.004615,0.016663,0.002883,0.002915,0.002218,0.006377,0.006338,0.004405,0.008170,0.003655,0.002963,0.003233,0.004113,0.002649,0.006633,0.002586,0.003092,0.004755,0.008078,0.008372,0.002707,0.004864,0.002184,0.004094,0.002090,0.004551,0.001848,0.002206,0.003171,0.003947,0.002345,0.004885,0.002594,0.007041,0.005736,0.003274,0.004072,0.003928,0.004460,0.008681,0.004841,0.001808,0.004759,0.002675,0.002242,0.002625,0.002841,0.005403,0.005920,0.002803,0.005253,0.003856,0.002477,0.006344,0.007048,0.005846,0.003187,0.006131,0.005706,0.003294,0.003858,0.006019,0.004970,0.003590,0.008780,0.002126,0.005341,0.004152,0.005130,0.006254,0.003832,0.001742,0.004580,0.006088,0.005514,0.005007,0.003218,0.002995,0.004273,0.003129,0.004710,0.007743,0.002893,0.005967,0.006204,0.003708,0.004267,0.007944,0.003336,0.002571,0.003035,0.004862,0.002942,0.004112,0.001919,0.008067
11,0.001445,0.002099,0.000806,0.002774,0.001852,0.002562,0.004670,0.002458,0.002291,0.002529,0.002352,0.001634,0.000640,0.001072,0.001826,0.003172,0.002474,0.003902,0.003411,0.001601,0.001622,0.002445,0.002464,0.001417,0.003665,0.001991,0.000663,0.001917,0.000956,0.000688,0.007245,0.000924,0.001146,0.001555,0.004299,0.002344,0.001121,0.004665,0.000652,0.001330,0.000472,0.001861,0.000728,0.000760,0.001998,0.001296,0.001610,0.001958,0.001335,0.003729,0.002891,0.001223,0.001489,0.001607,0.001718,0.003743,0.001663,0.001230,0.001405,0.001449,0.001769,0.001133,0.002418,0.001837,0.002326,0.002262,0.003308,0.002284,0.001251,0.003299,0.004387,0.003544,0.001672,0.001918,0.002037,0.001285,0.001493,0.002496,0.001209,0.001609,0.004821,0.001003,0.002341,0.001560,0.002393,0.002656,0.001619,0.001013,0.002152,0.002691,0.001486,0.001788,0.001224,0.000965,0.001204,0.001339,0.002119,0.002275,0.001195,0.003635,0.001859,0.002443,0.001382,0.002469,0.002030,0.000839,0.001271,0.002095,0.001518,0.001891,0.001123,0.003965
16,0.002168,0.002456,0.001581,0.002986,0.002213,0.003253,0.004303,0.002178,0.001841,0.003299,0.002363,0.003923,0.001702,0.002114,0.002539,0.002653,0.002831,0.003806,0.002137,0.001902,0.001629,0.002742,0.002465,0.002162,0.006047,0.002152,0.001239,0.002344,0.002127,0.002431,0.003436,0.001612,0.002205,0.001976,0.004228,0.002748,0.001477,0.004131,0.001774,0.002267,0.001275,0.002703,0.001308,0.001800,0.002026,0.002777,0.001783,0.002092,0.001784,0.003094,0.002886,0.003132,0.002563,0.002423,0.002020,0.003401,0.001461,0.001461,0.002030,0.002063,0.002289,0.001413,0.002510,0.001670,0.002804,0.003364,0.003708,0.002256,0.002166,0.002735,0.002815,0.003735,0.002981,0.001996,0.002094,0.002147,0.001559,0.002654,0.002217,0.002112,0.002367,0.001688,0.001896,0.002378,0.003085,0.003712,0.003618,0.001366,0.002965,0.002639,0.003431,0.002939,0.002458,0.001799,0.001518,0.002630,0.002738,0.002752,0.001753,0.003406,0.002687,0.002704,0.001949,0.002195,0.003410,0.002569,0.002137,0.001893,0.002131,0.002428,0.001548,0.003161
31,0.002195,0.002807,0.001599,0.004437,0.002256,0.003072,0.005401,0.002149,0.003997,0.003696,0.002341,0.003581,0.001335,0.001802,0.002007,0.001897,0.002201,0.005206,0.001827,0.002071,0.001471,0.002550,0.002127,0.001638,0.004563,0.001634,0.000883,0.001073,0.003748,0.001881,0.002581,0.001369,0.002386,0.002128,0.002666,0.002927,0.001557,0.001945,0.000841,0.001757,0.000410,0.002066,0.001285,0.001337,0.001742,0.002524,0.001936,0.002696,0.001466,0.004116,0.003129,0.002059,0.002323,0.002589,

In [7]:
qt_train = []
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()
for col in tqdm(colNames,total=len(colNames)):
    #print(col)
    qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')  # 将每个特征缩放在同样的范围或分布下。通过执行一个秩转换能够使异常的分布平滑化，并且能够比缩放更少的收到离群值的影响。
    train_nn[col] = qt.fit_transform(train_nn[[col]])
    test_nn[col] = qt.transform(test_nn[[col]])    
    qt_train.append(qt)

In [8]:
train_nn[['stock_id','time_id','target']] = train[['stock_id','time_id','target']]
test_nn[['stock_id','time_id']] = test[['stock_id','time_id']]
train_nn

,wap1_sum,wap1_std,wap2_sum,wap2_std,wap3_sum,wap3_std,wap4_sum,wap4_std,log_return1_realized_volatility,log_return2_realized_volatility,log_return3_realized_volatility,log_return4_realized_volatility,wap_balance_sum,wap_balance_amax,price_spread_sum,price_spread_amax,price_spread2_sum,price_spread2_amax,bid_spread_sum,bid_spread_amax,ask_spread_sum,ask_spread_amax,total_volume_sum,total_volume_amax,volume_imbalance_sum,volume_imbalance_amax,bid_ask_spread_sum,bid_ask_spread_amax,log_return1_realized_volatility_500,log_return2_realized_volatility_500,log_return3_realized_volatility_500,log_return4_realized_volatility_500,log_return1_realized_volatility_400,log_return2_realized_volatility_400,log_return3_realized_volatility_400,log_return4_realized_volatility_400,log_return1_realized_volatility_300,log_return2_realized_volatility_300,log_return3_realized_volatility_300,log_return4_realized_volatility_300,log_return1_realized_volatility_200,log_return2_realized_volatility_200,log_return3_realized_volatility_200,log_return4_realized_volatility_200,log_return1_realized_volatility_100,log_return2_realized_volatility_100,log_return3_realized_volatility_100,log_return4_realized_volatility_100,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_size_amax,trade_size_amin,trade_order_count_sum,trade_order_count_amax,trade_amount_sum,trade_amount_amax,trade_amount_amin,trade_tendency,trade_f_max,trade_f_min,trade_df_max,trade_df_min,trade_abs_diff,trade_energy,trade_iqr_p,trade_abs_diff_v,trade_energy_v,trade_iqr_p_v,trade_log_return_realized_volatility_500,trade_seconds_in_bucket_count_unique_500,trade_size_sum_500,trade_order_count_sum_500,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_sum_400,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_sum_300,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_sum_200,trade_log_return_realized_volatility_100,trade_seconds_in_bucket_count_unique_100,trade_size_sum_100,trade_order_count_sum_100,size_tau,size_tau2,size_tau_200,size_tau2_200,size_tau_300,size_tau2_300,size_tau_400,size_tau2_400,size_tau2_d,log_return1_realized_volatility_mean_stock,log_return1_realized_volatility_std_stock,log_return1_realized_volatility_max_stock,log_return1_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return1_realized_volatility_400_mean_stock,log_return1_realized_volatility_400_std_stock,log_return1_realized_volatility_400_max_stock,log_return1_realized_volatility_400_min_stock,log_return2_realized_volatility_400_mean_stock,log_return2_realized_volatility_400_std_stock,log_return2_realized_volatility_400_max_stock,log_return2_realized_volatility_400_min_stock,log_return1_realized_volatility_300_mean_stock,log_return1_realized_volatility_300_std_stock,log_return1_realized_volatility_300_max_stock,log_return1_realized_volatility_300_min_stock,log_return2_realized_volatility_300_mean_stock,log_return2_realized_volatility_300_std_stock,log_return2_realized_volatility_300_max_stock,log_return2_realized_volatility_300_min_stock,log_return1_realized_volatility_200_mean_stock,log_return1_realized_volatility_200_std_stock,log_return1_realized_volatility_200_max_stock,log_return1_realized_volatility_200_min_stock,log_return2_realized_volatility_200_mean_stock,log_return2_realized_volatility_200_std_stock,log_return2_realized_volatility_200_max_stock,log_return2_realized_volatility_200_min_stock,trade_log_return_realized_volatility_mean_stock,trade_log_return_realized_volatility_std_stock,trade_log_return_realized_volatility_max_stock,trade_log_return_realized_volatility_min_stock,trade_log_return_realized_volatility_400_

In [9]:
#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
from keras.backend import sigmoid,tanh,log,exp
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

def swish_2(x, beta = 0.95):
    return (x * sigmoid(beta * x))

def swish_3(x, beta = 1.05):
    return (x * sigmoid(beta * x))

def mish(x):
    return (x * tanh(log(1+exp(x))))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})
# get_custom_objects().update({'mish': Activation(mish)})

# hidden_units = (128,64,32)
# hidden_units = (512,128,32)
# hidden_units = (256,128,64,16)
# hidden_units = (512,128,64,16)
stock_embedding_size = 24

cat_data = train_nn['stock_id']

def base_model():
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(244,), name='num_data') # 247 - 1(stock_id) - 2(time_id, target) =244  # 258


    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    # Add one or more hidden layers
    hidden_units = (128,64,32)
    for n_hidden in hidden_units:
        out = keras.layers.Dense(n_hidden, activation='swish')(out)
#         out = keras.layers.Dense(n_hidden, activation='mish')(out)
    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model


def base_model_2(num_columns,  # 244
#                num_labels,  # output_dim
               hidden_units,   # (128,64,32)
               dropout_rates,  # list of dropout_rates
               stock_embedding_size = 24, 
               # ls = 1e-2,  # label_smoothing = ls
               lr = 1e-3):
    
    # input 
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(num_columns,), name='num_data')
    
    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, 
                                            stock_embedding_size,
                                            input_length=1, 
                                            name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    x = keras.layers.Concatenate()([stock_flattened, num_input])
    
    i=0
    # Add one or more hidden layers
    for n_hidden in hidden_units:
#         out = keras.layers.Dense(n_hidden, activation='swish')(out)
        x = keras.layers.Dense(n_hidden)(x)
#         if i%2 ==1:
#             x = keras.layers.BatchNormalization()(x)
        if i==0 or i==4:
            x = tf.keras.layers.GaussianNoise(0.1)(x)  # std of noise
        x = keras.layers.Activation('swish')(x)
#         x = keras.layers.Activation('swish')(x)
        x = keras.layers.Dropout(dropout_rates[i])(x)
        i+=1

    out = keras.layers.Dense(1, activation='linear', name='prediction')(x)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

In [10]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [11]:
# %%time
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2020)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train_nn)  # len -> 247

features_to_consider.remove('time_id')
features_to_consider.remove('target')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn1 = np.zeros(test_nn.shape[0])
valid_predictions_nn1 = np.zeros(train_nn.shape[0])

for n_count in range(n_folds):
    print('CV {}/{}'.format(counter, n_folds))
    
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0)
    
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]] # 按列连接两个矩阵，就是把两矩阵上下相加，要求列数相等。
    
    X_train = train_nn.loc[train_nn.time_id.isin(indexes), features_to_consider]
    y_train = train_nn.loc[train_nn.time_id.isin(indexes), target_name]
    X_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), features_to_consider]
    y_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), target_name]
    
    #############################################################################################
    # NN
    #############################################################################################
#     model = base_model()
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006),
#         keras.optimizers.Adam(learning_rate=0.006,decay=3e-5),
        loss=root_mean_squared_per_error
    )
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    num_data = X_train[features_to_consider]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    num_data = scaler.fit_transform(num_data.values)
    cat_data = X_train['stock_id']
    target =  y_train
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']
    
    model.fit([cat_data, num_data],
              target, 
              batch_size=2048, 
              epochs=1000, 
#               sample_weight= 1/np.square(y_train),
              validation_data=([cat_data_test, num_data_test], y_test), 
              callbacks=[es, plateau], 
              validation_batch_size=len(y_test), 
              shuffle=True, 
              verbose = 1)
#     preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    valid_predictions_nn1[train_nn.time_id.isin(values[n_count])] = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
        
    
#     score = round(rmspe(y_true = y_test, y_pred = preds),5)
    score = round(rmspe(y_true = y_test, y_pred = valid_predictions_nn1[train_nn.time_id.isin(values[n_count])]),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)

    

    tt =scaler.transform(test_nn[features_to_consider].values)
    #test_nn[target_name] += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)
    test_predictions_nn1 += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds
    #test[target_name] += model.predict([test['stock_id'], test[features_to_consider]]).reshape(1,-1)[0].clip(0,1e10)
    
    counter += 1
    features_to_consider.append('stock_id')
    
print('avg val loss: ', round(np.mean(scores_folds['NN']), 5))

CV 1/5
Epoch 1/1000
168/168 [==============================] - 3s 11ms/step - loss: 7.0904 - val_loss: 0.8210
Epoch 2/1000
168/168 [==============================] - 2s 11ms/step - loss: 0.4817 - val_loss: 0.3130
Epoch 3/1000
168/168 [==============================] - 2s 11ms/step - loss: 0.7037 - val_loss: 0.4798
Epoch 4/1000
168/168 [==============================] - 2s 10ms/step - loss: 0.6312 - val_loss: 0.7083
Epoch 5/1000
168/168 [==============================] - 2s 11ms/step - loss: 0.5860 - val_loss: 0.6089
Epoch 6/1000
168/168 [==============================] - 2s 10ms/step - loss: 0.5645 - val_loss: 0.6217
Epoch 7/1000
168/168 [==============================] - 2s 10ms/step - loss: 0.5024 - val_loss: 0.2337
Epoch 8/1000
168/168 [==============================] - 2s 12ms/step - loss: 0.2664 - val_loss: 0.2926
Epoch 9/1000
168/168 [==============================] - 2s 10ms/step - loss: 0.2711 - val_loss: 0.3164
Epoch 10/1000
168/168 [==============================] - 2s 11ms/s

168/168 [==============================] - 1s 8ms/step - loss: 0.7993 - val_loss: 0.6534
Epoch 3/1000
168/168 [==============================] - 1s 8ms/step - loss: 0.8161 - val_loss: 0.9195
Epoch 4/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.5939 - val_loss: 0.3205
Epoch 5/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.3511 - val_loss: 0.9055
Epoch 6/1000
168/168 [==============================] - 1s 7ms/step - loss: 2.6352 - val_loss: 0.3757
Epoch 7/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2763 - val_loss: 0.2455
Epoch 8/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2512 - val_loss: 0.2335
Epoch 9/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2471 - val_loss: 0.2324
Epoch 10/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2574 - val_loss: 0.2429
Epoch 11/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2548 - val_loss

168/168 [==============================] - 1s 7ms/step - loss: 0.2595 - val_loss: 0.2937
Epoch 15/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2319 - val_loss: 0.2243
Epoch 16/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2298 - val_loss: 0.2476
Epoch 17/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2366 - val_loss: 0.2472
Epoch 18/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2302 - val_loss: 0.2779
Epoch 19/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2357 - val_loss: 0.3494
Epoch 20/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2388 - val_loss: 0.2134
Epoch 21/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2410 - val_loss: 0.2858
Epoch 22/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2377 - val_loss: 0.2201
Epoch 23/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2413 - v

168/168 [==============================] - 1s 7ms/step - loss: 0.2060 - val_loss: 0.2170
Epoch 27/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2050 - val_loss: 0.2180
Epoch 28/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2050 - val_loss: 0.2175
Epoch 29/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2050 - val_loss: 0.2172
Epoch 30/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2049 - val_loss: 0.2168
Epoch 31/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2050 - val_loss: 0.2173
Epoch 32/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2049 - val_loss: 0.2171
Epoch 33/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2049 - val_loss: 0.2176
Epoch 34/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2050 - val_loss: 0.2181
Epoch 35/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2047 - v

168/168 [==============================] - 1s 8ms/step - loss: 0.2075 - val_loss: 0.2186
Epoch 41/1000
168/168 [==============================] - 1s 8ms/step - loss: 0.2062 - val_loss: 0.2155
Epoch 42/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2058 - val_loss: 0.2155
Epoch 43/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2057 - val_loss: 0.2155
Epoch 44/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2055 - val_loss: 0.2155
Epoch 45/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2056 - val_loss: 0.2149
Epoch 46/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2055 - val_loss: 0.2159
Epoch 47/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2055 - val_loss: 0.2153
Epoch 48/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2055 - val_loss: 0.2156
Epoch 49/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2054 - v

In [12]:
seed(41)
tf.random.set_seed(41)

target_name='target'
scores_folds = {}
model_name = 'NN2'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2021)
scores_folds[model_name] = []
counter = 1

# features_to_consider = list(train1)
features_to_consider = list(train_nn)

features_to_consider.remove('time_id')
features_to_consider.remove('target')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


# train1[features_to_consider] = train1[features_to_consider].fillna(train1[features_to_consider].mean())
# test1[features_to_consider] = test1[features_to_consider].fillna(train1[features_to_consider].mean())

# train1[pred_name] = 0
# test1[target_name] = 0
# test_predictions_nn2 = np.zeros(test_nn.shape[0])

train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn2 = np.zeros(test_nn.shape[0])
valid_predictions_nn2 = np.zeros(train_nn.shape[0])

for n_count in range(n_folds):
    print('CV {}/{}'.format(counter, n_folds))
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0)
    
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]] # 按列连接两个矩阵，就是把两矩阵上下相加，要求列数相等。
    
    X_train = train_nn.loc[train_nn.time_id.isin(indexes), features_to_consider]
    y_train = train_nn.loc[train_nn.time_id.isin(indexes), target_name]
    X_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), features_to_consider]
    y_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), target_name]
    
#     X_train = train1.loc[train1.time_id.isin(indexes), features_to_consider]
#     y_train = train1.loc[train1.time_id.isin(indexes), target_name]
#     X_test = train1.loc[train1.time_id.isin(values[n_count]), features_to_consider]
#     y_test = train1.loc[train1.time_id.isin(values[n_count]), target_name]
    
    #############################################################################################
    # NN
    #############################################################################################
    
    model = base_model_2(244,
                       hidden_units=[512,256,128,128,64,64,32,16],
                       # hidden_units=[128,128,128,128,32],   #[256,128,64]
                       dropout_rates=[0.02,0.02,0.01,0.01,0.01,0.02,0.02,0.02]
                       # dropout_rates=[0.2,0.2,0.2,0.2,0.2,0.2,]
                      )
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006),
        loss=root_mean_squared_per_error
    )
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    num_data = X_train[features_to_consider]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    
    
    cat_data = X_train['stock_id']    
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']
    model.fit([cat_data, num_data], 
              target,               
              batch_size=2048,
              epochs=1000,
              validation_data=([cat_data_test, num_data_test], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
              verbose = 1
             )
    
#     preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    
    valid_predictions_nn2[train_nn.time_id.isin(values[n_count])] = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]

#     score = round(rmspe(y_true = y_test, y_pred = preds),5)
    score = round(rmspe(y_true = y_test, y_pred = valid_predictions_nn2[train_nn.time_id.isin(values[n_count])]),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)
    
    
    
    tt =scaler.transform(test_nn[features_to_consider].values)
    #test_nn[target_name] += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)
#     test_predictions_nn2 += model.predict([test1['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds
    test_predictions_nn2 += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds
    #test[target_name] += model.predict([test['stock_id'], test[features_to_consider]]).reshape(1,-1)[0].clip(0,1e10)
    
    counter += 1
    features_to_consider.append('stock_id')
    
print('avg val loss: ', round(np.mean(scores_folds['NN2']), 5))

CV 1/5
Epoch 1/1000
168/168 [==============================] - 11s 58ms/step - loss: 2.3603 - val_loss: 0.4625
Epoch 2/1000
168/168 [==============================] - 10s 57ms/step - loss: 0.6216 - val_loss: 0.4078
Epoch 3/1000
168/168 [==============================] - 10s 57ms/step - loss: 0.4908 - val_loss: 0.3805
Epoch 4/1000
168/168 [==============================] - 10s 57ms/step - loss: 0.2970 - val_loss: 0.2324
Epoch 5/1000
168/168 [==============================] - 10s 57ms/step - loss: 0.4004 - val_loss: 0.2209
Epoch 6/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.3236 - val_loss: 0.3767
Epoch 7/1000
168/168 [==============================] - 10s 57ms/step - loss: 0.2912 - val_loss: 0.2177
Epoch 8/1000
168/168 [==============================] - 10s 57ms/step - loss: 0.2433 - val_loss: 0.2166
Epoch 9/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.2468 - val_loss: 0.2181
Epoch 10/1000
168/168 [==============================] - 

Epoch 5/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.2798 - val_loss: 0.2297
Epoch 6/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.2663 - val_loss: 0.2255
Epoch 7/1000
168/168 [==============================] - 10s 60ms/step - loss: 0.2598 - val_loss: 0.2799
Epoch 8/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.2492 - val_loss: 0.2338
Epoch 9/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.2420 - val_loss: 0.2278
Epoch 10/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.2358 - val_loss: 0.2960
Epoch 11/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.2335 - val_loss: 0.2257
Epoch 12/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2325 - val_loss: 0.2226
Epoch 13/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.2320 - val_loss: 0.2186
Epoch 14/1000
168/168 [==============================] - 10s

Epoch 32/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2076 - val_loss: 0.2091
Epoch 33/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2060 - val_loss: 0.2083
Epoch 34/1000
168/168 [==============================] - 10s 60ms/step - loss: 0.2053 - val_loss: 0.2080
Epoch 35/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2052 - val_loss: 0.2092
Epoch 36/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2051 - val_loss: 0.2099
Epoch 37/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2052 - val_loss: 0.2097
Epoch 38/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2052 - val_loss: 0.2078
Epoch 39/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2050 - val_loss: 0.2100
Epoch 40/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.2049 - val_loss: 0.2087
Epoch 41/1000
168/168 [==============================] 

Epoch 52/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.1941 - val_loss: 0.2174
Epoch 53/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.1940 - val_loss: 0.2180
Epoch 54/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.1937 - val_loss: 0.2182
Epoch 55/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.1940 - val_loss: 0.2179
Fold 4 NN2: 0.21194
CV 5/5
Epoch 1/1000
168/168 [==============================] - 11s 59ms/step - loss: 1.7927 - val_loss: 0.2385
Epoch 2/1000
168/168 [==============================] - 10s 58ms/step - loss: 0.3888 - val_loss: 0.2297
Epoch 3/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.3391 - val_loss: 0.4426
Epoch 4/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.3168 - val_loss: 0.6333
Epoch 5/1000
168/168 [==============================] - 10s 59ms/step - loss: 0.4053 - val_loss: 0.2915
Epoch 6/1000
168/168 [===========

In [13]:
valid_predictions_nn1, valid_predictions_nn2

(array([0.00372068, 0.00139473, 0.00230175, ..., 0.00327022, 0.00460468,
        0.002109  ]),
 array([0.0037984 , 0.00149692, 0.00218644, ..., 0.00323129, 0.0054737 ,
        0.00209903]))

In [14]:
test_predictions_nn1, test_predictions_nn2

(array([0.00130807, 0.00250945, 0.00250945]),
 array([0.00171968, 0.00283447, 0.00283447]))

In [15]:
# pd.Series(valid_predictions_nn1).to_csv('nn1_valid_pred_knn.csv',index=0)
# pd.Series(valid_predictions_nn2).to_csv('nn2_valid_pred_knn.csv',index=0)

In [16]:
import gc
del train_nn, test_nn
del X_train, X_test
del cat_data, num_data, cat_data_test, num_data_test
gc.collect()

9343